# Example Workflow for Normalizing and Validating δ15N Data from IRMS


In [20]:
from isotools.processors import NitrogenProcessor
from isotools.calibration import Calibrator
from isotools.strategies import TwoPointStrategy
from isotools.reporting import aggregate_samples, Reporter
from isotools.standards import USGS32_N2, USGS34_N2

In [21]:
# 1. Setup
proc = NitrogenProcessor(exclude_rows=[26])
calib = Calibrator(TwoPointStrategy())

In [22]:
# 2. Load Raw Data (No aggregation yet!)
raw_df = proc.load_data(r"DATA\nitrate 26112025.xls")
# raw_df has 100 rows for 20 samples (5 replicates each)

raw_df.head()

,row,sample_name,sample_id_2,amount,comment,peak_nr,amp_28,amp_29,area_28,area_29,r15n,d15n
3,10,nitrato ES3,NaN,0.558,NaN,2,5055,3592,71.990,0.511,0.003622,-15.346
5,11,nitrato ES3,NaN,0.588,NaN,2,5445,3867,78.306,0.556,0.003622,-15.414
7,12,nitrato ES3,NaN,0.592,NaN,2,5314,3774,77.983,0.554,0.003622,-15.287
9,13,nitrato ES3,NaN,0.559,NaN,2,4891,3474,71.222,0.506,0.003622,-15.412
11,14,nitrato ES6,NaN,0.487,NaN,2,5684,4128,83.973,0.610,0.003702,6.575


In [23]:
# 3. Calibrate Raw Data
# Fits standards internally, then corrects all 100 rows
calibrated_df = calib.calibrate(
    raw_df, standards=[USGS32_N2, USGS34_N2], target_col="d15n"
)

calibrated_df.head()

,row,sample_name,sample_id_2,amount,comment,peak_nr,amp_28,amp_29,area_28,area_29,r15n,d15n,corrected_delta
3,10,nitrato ES3,NaN,0.558,NaN,2,5055,3592,71.990,0.511,0.003622,-15.346,-22.194977
5,11,nitrato ES3,NaN,0.588,NaN,2,5445,3867,78.306,0.556,0.003622,-15.414,-22.262545
7,12,nitrato ES3,NaN,0.592,NaN,2,5314,3774,77.983,0.554,0.003622,-15.287,-22.136351
9,13,nitrato ES3,NaN,0.559,NaN,2,4891,3474,71.222,0.506,0.003622,-15.412,-22.260558
11,14,nitrato ES6,NaN,0.487,NaN,2,5684,4128,83.973,0.610,0.003702,6.575,-0.413109


In [24]:
# TODO:4. Diagnostics (Optional but recommended)

In [25]:
# 5. Summarize (Aggregation)
stats_df = aggregate_samples(calibrated_df)
stats_df.head()

,row_mean,row_sem,row_count,sample_id_2_mean,sample_id_2_sem,sample_id_2_count,amount_mean,amount_sem,amount_count,comment_mean,...,area_29_count,r15n_mean,r15n_sem,r15n_count,d15n_mean,d15n_sem,d15n_count,corrected_delta_mean,corrected_delta_sem,corrected_delta_count
sample_name,,,,,,,,,,,,,,,,,,,,,
USGS32,19.5,0.645497,4,NaN,0.0,0,0.57650,0.001936,4,NaN,...,4,0.004370,6.774646e-07,4,188.14050,0.182134,4,180.000000,0.180978,4
USGS34,23.5,0.645497,4,NaN,0.0,0,0.48150,0.002901,4,NaN,...,4,0.003697,8.660254e-08,4,5.17925,0.023859,4,-1.800000,0.023707,4
USGS35,28.0,0.577350,3,NaN,0.0,0,0.47900,0.003606,3,NaN,...,3,0.003714,5.773503e-08,3,9.80700,0.015716,3,2.798378,0.015616,3
nitrato ES3,11.5,0.645497,4,NaN,0.0,0,0.57425,0.009132,4,NaN,...,4,0.003622,1.224745e-07,4,-15.36475,0.030352,4,-22.213608,0.030159,4
nitrato ES6,15.5,0.645497,4,NaN,0.0,0,0.47875,0.003473,4,NaN,...,4,0.003698,3.181981e-06,4,5.32700,0.866005,4,-1.653188,0.860509,4


In [26]:
# 6. Report
reporter = Reporter(decimals=2)
final_table = reporter.create_report(stats_df, target_col="d15n")
print(final_table)

             Delta d15n (Air)  Error (1s)  N
sample_name                                 
USGS32                 180.00        0.18  4
USGS34                  -1.80        0.02  4
USGS35                   2.80        0.02  3
nitrato ES3            -22.21        0.03  4
nitrato ES6             -1.65        0.86  4
